In [ ]:
import os
import pickle
import shapely
from pathlib import Path
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray as rxa
import matplotlib.pyplot as plt

# Add main repo to path
import sys
from os.path import expanduser
sys.path.append(expanduser('../../'))

from spicy_snow.IO.user_dates import get_input_dates

from spicy_snow.retrieval import retrieval_from_parameters, retrieve_snow_depth

# import functions for downloading
from spicy_snow.download.sentinel1 import s1_img_search, hyp3_pipeline, download_hyp3, combine_s1_images
from spicy_snow.download.forest_cover import download_fcf
from spicy_snow.download.snow_cover import download_snow_cover

# import functions for pre-processing
from spicy_snow.processing.s1_preprocessing import merge_partial_s1_images, s1_orbit_averaging,\
s1_clip_outliers, subset_s1_images, ims_water_mask, s1_incidence_angle_masking, merge_s1_subsets

# import the functions for snow_index calculation
from spicy_snow.processing.snow_index import calc_delta_VV, calc_delta_cross_ratio, \
    calc_delta_gamma, clip_delta_gamma_outlier, calc_snow_index, calc_snow_index_to_snow_depth

# import the functions for wet snow flag
from spicy_snow.processing.wet_snow import id_newly_frozen_snow, id_newly_wet_snow, \
    id_wet_negative_si, flag_wet_snow

# setup root logger
from spicy_snow.utils.spicy_logging import setup_logging

In [ ]:
work_dir = Path('~/scratch/broken-combo/').expanduser()
os.makedirs(work_dir, exist_ok = True)

dates = get_input_dates('2021-02-20', '2021-02-18')
area = shapely.geometry.box(-117, 44, -116, 45)

out_nc = Path(f'~/scratch/broken-combo/spicy-lowman-v1.nc').expanduser()

search_results = s1_img_search(area, dates)
jobs = hyp3_pipeline(search_results, job_name = 'broken-combo', existing_job_name = 'broken-combo')

In [ ]:
imgs = download_hyp3(jobs, area, outdir = work_dir.joinpath('tmp'), clean = False)

In [ ]:
ds = combine_s1_images(imgs)

In [ ]:
ds = merge_partial_s1_images(ds)

In [ ]:
ds.sel(band = "inc").isel(time = 0)['s1'].plot()